<img src="images/ublogo.png"/>

### CSE610 - Bayesian Non-parametric Machine Learning

  - Lecture Notes
  - Instructor - Varun Chandola
  - Term - Fall 2020

In [4]:
import numpy as np
from scipy.stats import multivariate_normal as mvn
from scipy.stats import zscore,invwishart,dirichlet,multinomial,norm
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import GPy
import time
import pods
from sklearn.datasets import make_classification, openml
from scipy.spatial.distance import pdist,squareform,cdist
sns.set(color_codes=True)
sns_c = sns.color_palette(palette='deep')
sns.set_style('whitegrid')
sns.set_context('paper',font_scale=2)
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
#%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
%matplotlib inline

In [ ]:
def bayesPosterior(X,y,sigmansq,Sigmap):
    Ainv = np.linalg.inv((1/sigmansq)*np.dot(X,X.T) + np.linalg.inv(Sigmap))
    wpost = (1/sigmansq)*np.dot(Ainv,np.dot(X,y))
    return wpost,Ainv
    

In [ ]:
def plot_contours(ax,mean,cov,limits=(-10,10),cm='Reds',levels=6):
    w1s = w2s = np.linspace(limits[0],limits[1],100)
    W1,W2 = np.meshgrid(w1s,w2s)

    pdfs = []
    for w1,w2 in zip(W1.flatten(),W2.flatten()):
        w = np.array([w1,w2])
        pdfs.append(mvn.pdf(w,mean,cov))
    pdfs = np.array(pdfs)
    pdfs = np.reshape(pdfs,W1.shape)

    cfset = ax.contourf(W1, W2, pdfs, levels=levels,cmap=cm)
    cset = ax.contour(W1, W2, pdfs, levels=levels, colors='k',alpha=0.6)
    ax.clabel(cset, inline=1, fontsize=10)

In [ ]:
'''
Copied from: https://github.com/krasserm/bayesian-machine-learning/
'''
def plot_gp(mu, cov, X, X_train=None, Y_train=None, samples=[],legend=True):
    X = X.ravel()
    mu = mu.ravel()
    uncertainty = 1.96 * np.sqrt(np.diag(cov))
    
    plt.fill_between(X, mu + uncertainty, mu - uncertainty, alpha=0.1)
    plt.plot(X, mu, label='Mean')
    for i, sample in enumerate(samples):
        plt.plot(X, sample, lw=1, ls='--', label=f'Sample {i+1}')
    if X_train is not None:
        plt.plot(X_train, Y_train, 'rx')
    if legend:
        plt.legend(ncol=3)

In [ ]:
def load_mauna_loa_atmospheric_co2():
    ml_data = openml.fetch_openml(data_id=41187)
    months = []
    ppmv_sums = []
    counts = []

    y = ml_data.data[:, 0]
    m = ml_data.data[:, 1]
    month_float = y + (m - 1) / 12
    ppmvs = ml_data.target

    for month, ppmv in zip(month_float, ppmvs):
        if not months or month != months[-1]:
            months.append(month)
            ppmv_sums.append(ppmv)
            counts.append(1)
        else:
            # aggregate monthly sum to produce average
            ppmv_sums[-1] += ppmv
            counts[-1] += 1

    months = np.asarray(months).reshape(-1, 1)
    avg_ppmvs = np.asarray(ppmv_sums) / counts
    return months, avg_ppmvs




In [ ]:
def plot_2outputs(m,xlim,ylim1,ylim2):
    fig = plt.figure(figsize=(12,8))
    #Output 1
    ax1 = fig.add_subplot(211)
    ax1.set_xlim(xlim)
    ax1.set_title('Output 1')
    m.plot(plot_limits=xlim,fixed_inputs=[(1,0)],which_data_rows=slice(0,100),ax=ax1)
    ax1.plot(Xt1[:,:1],Yt1,'rx',mew=1.5)
    ax1.set_ylim(ylim1)
    #Output 2
    ax2 = fig.add_subplot(212)
    ax2.set_xlim(xlim)
    ax2.set_title('Output 2')
    m.plot(plot_limits=xlim,fixed_inputs=[(1,1)],which_data_rows=slice(100,200),ax=ax2)
    ax2.plot(Xt2[:,:1],Yt2,'rx',mew=1.5)
    ax2.set_ylim(ylim2)

In [ ]:
def normalinvwishartsample(params):
    '''
    Generate sample from a Normal Inverse Wishart distribution

    Inputs:
    params - Parameters for the NIW distribution 
        mu     - Mean parameter: n x 1 numpy array
        lmbda  - Scalar parameter for normal distribution covariance matrix
        Psi    - Precision parameter: d x d numpy array
        nu     - Scalar parameter for Wishart distribution

    Output:
    Sample - Sample mean vector, mu_s and Sample covariance matrix, W_s
    '''
    mu,lmbda,Psi,nu = params
    # first sample W from a Inverse Wishart distribution
    W_s = invwishart(df=nu, scale=Psi).rvs()
    mu_s = np.random.multivariate_normal(mu.flatten(),W_s/lmbda,1) 
    return np.transpose(mu_s),W_s

In [1]:
def normalinvwishartposterior(X,params):
    '''
    Posterior distribution for Gaussian likelihood and Normal Inverse Wishart prior

    Inputs:
    X     - Dataset matrix: n x d numpy array
    params - Parameters for the NIW distribution 
        mu    - Mean parameter: n x 1 numpy array
        lmbda - Scalar parameter for normal distribution covariance matrix
        Psi   - Precision parameter: d x d numpy array
        nu    - Scalar parameter for Wishart distribution

    Output:
    Posterior distribution parameters - tuple (mu_n, lmbda_n,Psi_n, nu_n)
    '''
    mu,lmbda,Psi,nu = params

    n = X.shape[0]
    d = X.shape[1]
    X_mean = np.mean(X,axis=0)
    X_mean = X_mean[:,np.newaxis]
    mu_n = (lmbda*mu + n*X_mean)/(lmbda+n)
    S = scatter(X)
    Psi_n = Psi + S + ((lmbda*n)/(lmbda+n))*np.dot(mu - X_mean,np.transpose(mu - X_mean))

    nu_n = nu + n
    lmbda_n = lmbda + n
    Psi_s = invwishart(df=nu_n, scale=Psi_n).rvs()
    den = nu_n - d + 1
    mu_s = multivariatet(mu_n.flatten(),Psi_n/(lmbda_n*den),den,1)
    return (np.transpose(mu_s), Psi_s)



In [ ]:
def plotClusters(samples,thetas=None,z=None):
    if thetas == None:
        plt.scatter(samples[:,0],samples[:,1])
    else:
        thetameans = []
        K = len(thetas)
        for k in range(K):
            thetameans.append(thetas[k][0])
        thetameans = np.array(thetameans)
        for k in range(K):
            plt.scatter(samples[z == k,0],samples[z == k,1])
        plt.legend([str(k) for k in range(K)])
        plt.scatter(thetameans[:,0],thetameans[:,1],marker='x')
        for k in range(K):
            plt.text(thetameans[k,0],thetameans[k,1],str(k))

In [ ]:
def scatter(x):
    return np.dot(np.transpose(x - np.mean(x,0)),x - np.mean(x,0))

In [ ]:
def multivariatet(mu,Sigma,N,M):
    '''
    Output:
    Produce M samples of d-dimensional multivariate t distribution
    Input:
    mu = mean (d dimensional numpy array or scalar)
    Sigma = scale matrix (dxd numpy array)
    N = degrees of freedom
    M = # of samples to produce
    '''
    d = len(Sigma)
    g = np.tile(np.random.gamma(N/2.,2./N,M),(d,1)).T
    Z = np.random.multivariate_normal(np.zeros(d),Sigma,M)
    return mu + Z/np.sqrt(g)